## 糖尿病诊断结果的预测

---

#### 介绍

本实验以概率论和统计学相关知识为基础，采用实例的方式讲解了朴素贝叶斯算法的具体实现流程。并且使用该算法完成了通过血压血糖等指标预测患者是否具有糖尿病的实验。

#### 知识点

- 条件概率
- 贝叶斯推断
- 正态分布函数

---

###  数据集的介绍


挑战开始之前，让我们先将本挑战需要用到的数据下载下来：

In [ ]:
!wget https://labfile.oss.aliyuncs.com/courses/2316/diabetes.csv

首先，让我们来加载一下这些数据：

In [ ]:
import pandas as pd
df = pd.read_csv("diabetes.csv")
df

从上面可以看出，该数据集中一共有 768 条数据。 每条数据含有 8 个特征和 1 个标签。前 8 列为病人的各项指标特征：

| Pregnancies 	| Glucose 	| BloodPressure 	| SkinThickness 	| Insulin 	| BMI 	| DiabetesPedigreeFunction 	| Age 	|
|:-----------:	|:------------:	|:-------------:	|:-------------:	|:-------:	|:------------:	|:------------------------:	|:----:	|
| 怀孕次数 	| 葡萄糖测试值 	| 血压 	| 皮肤厚度 	| 胰岛素 	| 身体质量指数 	| 糖尿病遗传函数 	| 年龄 	|

最后一列 Outcome 为每条数据的糖尿病标签，其中 1 表示有糖尿病 ，0 表示没有糖尿病。

我们的目的就是找到糖尿病与这些特征之间的关系，进而使我们能够预测任何特征值下的病人是否患有糖尿病。

将这 8 个特征取值记为 $A_1,A_2,...,A_8$，将是否患有糖尿病记作 $y$。

那么我们的目的就是计算 $P(y=1|(A_1A_2,...A_8))$ 和  $P(y=0|(A_1A_2,...A_8))$。若前者的概率大于后者，则表示当特征值为 $A_1,A_2,...,A_8$ 时，该患者患有糖尿病。反之，则该病人没有糖尿病。

这里我们可以使用朴素贝叶斯算法对上面问题进行求解。

在进行问题的求解之前，让我先来将上面数据集的标签与特征分开：

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

df.columns = ['one', 'two', 'three', 'four',
              'five', 'six', 'seven', 'eight', 'target']
# 为了方便计算，这里我们只用 700 条数据
data = np.array(df.iloc[:700, :])
# 将数据分开
X, y = data[:, :-1], data[:, -1]
# 将原数据随机划分成 20% 的测试集和 80% 的训练集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=3, stratify=y)
data.shape, X_train.shape, X_test.shape

### 贝叶斯方法预测糖尿病

#### 朴素贝叶斯算法的数学推导

在本实验之前，如果你并不了解朴素贝叶斯方法也不必惊慌。其实朴素贝叶斯就是对上一个试验中的概率论三大公式的应用。你可以通过下面实例学习朴素贝叶斯算法的推导步骤。

朴素贝叶斯算法的核心思想就是利用上个实验中所提到的条件概率和贝叶斯公式来推断概率  $P(y=1|(A_1A_2,...A_8))$ 和  $P(y=0|(A_1A_2,...A_8))$ ，然后将其进行比较，得到预测结果的过程。

条件概率公式：

$$P(B|A)=\frac{P(AB)}{P(A)}$$

贝叶斯公式：

$$P(A|B)=\frac{P(AB)}{P(B)}=\frac{P(A)P(B|A)}{P(B)}$$

接下来，让我们利用这两个公式计算 $P(y=1|(A_1A_2,...A_8))$ 和  $P(y=0|(A_1A_2,...A_8))$ 的概率，这里统一记作 $P(y_i|(A_1A_2,...A_8))$。其中 $y_i$ 的可取值为 {0,1}。  

利用贝叶斯公式可以将目标概率转换为：
    $$P(y_i|A_1,A_2,...,A_8) =\frac{ P(y_i)P((A_1A_2...A_8)|y_i)}{p(A_1A_2...A_8)}$$

因此，我们的目的就是比较：$\frac{ P(y_0)P((A_1A_2...A_8)|y_i)}{p(A_1A_2...A_8)}$ 和 $\frac{ P(y_1)P((A_1A_2...A_8)|y_i)}{p(A_1A_2...A_8)} $ 的值。

由于我们只需要知道它们谁大谁小，不必知道具体的值。因此，我们可以划去两边相等的变量。首先，我们可以将两边的分母划去。其次，由于一般的先验概率都是均匀分布的，即患有糖尿病和不患糖尿病的人数相等，即 $p(y_0)=p(y_1)$，那么就可以将其划去。本实验将 $p(y_0)$ 和 $p(y_1)$ 划去，是由于概率编程中，常常把变量的先验设置为均匀分布，即概率相等。当然，你也可以尝试不划去这两个变量。

综上，我们只需要比较 $P((A_1A_2...A_8)|y_0)$ 和 $P((A_1A_2...A_8)|y_1)$ 的值，就能够得到 $P(y_0|(A_1A_2,...A_8))$ 和 $ P(y_1|(A_1A_2,...A_8))$ 谁大谁小，进而判断出该患者是否患有糖尿病。 

由于这些特征相互独立，则有：

 
$$  P((A_1A_2...A_8)|y_i) = P(A_1|y_i)\cdot P(A_2|y_i)\cdot P(A_3|y_i)\cdot P(A_4|y_i)\cdot P(A_5|y_i)\cdot P(A_6|y_i)\cdot P(A_7|y_i)\cdot  P(A_8|y_i)   $$


综上可知，如果需要判断特征为 $（A_1,A_2,...,A_8）$ 的患者是否患有糖尿病 。那么我们只需要知道   $P(A_1|y_i)，P(A_2|y_i)....，P(A_8|y_i)$ 的值即可，其中 $i=1,2$ 。那么如何求取 $P(A_1|y_i)，P(A_2|y_i)....，P(A_8|y_i)$ 的值呢？

#### 朴素贝叶斯的代码实现

在朴素贝叶斯分类器中，当需要估计第 $k$ 个属性在第 $i$ 类中出现的概率 $p(A_k|y_i)$ 时，如果变量为离散型变量则直接统计其概率即可。若变量是连续型变量。可以假定其概率分布然后根据分布函数估计其概率值。

拿求取 $p(A_1=3|y=1)$ 举例。对于这种连续型变量的求法如下： 

- 得到数据集中 $y=1$ 的数据。
- 假设 $A_1$ 在这些数据中呈正态分布。
- 根据已经统计的数据，估计该分布的均值和标准差，得到具体的分布函数。
- 有了分布函数后，将 $A_1=3$ 带入密度函数，得到的值就为  $p(A_1=3|y=1)$ 了。

综上，为了求取  $p(A_k|y_i)$ 的值，我们需要知道在数据集 $y_i$ 下 $A_k$ 的均值和标准差，进而得到所对应的概率密度函数。最后将 $A_k$ 的具体值带入该密度函数中，就可以得到相应的概率 $p(A_k|y_i)$ 了。

首先让我们将数据集 $y=1$ 和   $y=0$ 分开：

In [ ]:
labels = [0, 1]
data = {label: [] for label in labels}
for f, label in zip(X_train, y_train):
    data[label].append(f)
# 输出
print("统计数据中 y=0 的样本数：", len(data[0]))
print("统计数据中 y=1 的样本数：", len(data[1]))

接下来，让我们先定义一下标准差和均值的求取函数：

In [ ]:
import math

# 均值的计算公式
def mean(X):
    return sum(X) / float(len(X))

# 标准差
def stdev(X):
    avg = mean(X)
    return math.sqrt(sum([pow(x-avg, 2) for x in X]) / float(len(X)))

#测试函数
test = [1,2,3,4,5]
mean(test),stdev(test)

由于我们需要分别求出 y=0 和 y = 1 下的 8 个特征的均值和标准差。因此，让我们来写一个函数对其进行统一处理：

In [ ]:
# zip函数将数据实例中每个属性的值分组到它们自己的列表中，以便我们可以计算属性的均值和标准差值。
def summarize(train_data):
    summaries = [(mean(i), stdev(i)) for i in zip(*train_data)]
    return summaries

# 分别计算 y=0 和 y = 1 下的 8 个特征的均值和标准差
model = {0: summarize(data[0]), 1: summarize(data[1])}
model

从上面的结果可以看出，我们已经得到了  y=0 和 y = 1 下的 8 个特征的均值和标准差。其实到这里，我们的贝叶斯模型已经出来了，上面的 `model` 就是我们的朴素贝叶斯模型。

通过上面的均值和标准差我们能够得到 y = 0 时的每个特征的概率密度函数，进而得到在 y = 0  下， $A_1,...,A_8$ 的概率值，即$P(A_1|y_0)，P(A_2|y_0)....，P(A_8|y_0)$ 的值。  

举个例子 ，当 $(A_1,A_2,...,A_8) = (1,2,...8)$ 时，$P(A_1|y_0)，P(A_2|y_0)....，P(A_8|y_0)$ 的分别值为：

In [ ]:
# 呈正态分布的概率密度函数
# 传入均值标准差和 x 的值，返回 x 对应的概率
def gaussian_probality(x, mean, stdev):
    # 传正态分布的计算公式
    exponent = math.exp(-(math.pow(x-mean, 2)/(2*math.pow(stdev, 2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent


test = (1, 2, 3, 4, 5, 6, 7, 8)
p0 = 1
for i in range(8):
    mean = model[0][i][0]
    stdev = model[0][i][1]
    p = gaussian_probality(test[i], mean, stdev)
    p0 = p*p0
    print("p(A{}={}|y=0)={}".format(i+1, test[i], p))
print("p(A1,A2..A8|y=0)={}".format(p0))

同理我们也可以计算 $P(A_1|y_1)，P(A_2|y_1)....，P(A_8|y_1)$ 的值为：

In [ ]:
p1 = 1
for i in range(8):
    mean = model[1][i][0]
    stdev = model[1][i][1]
    p = gaussian_probality(test[i], mean, stdev)
    p1 = p1*p
    print("p(A{}={}|y=0)={}".format(i+1, test[i], p))
print("p(A1,A2..A8|y=1)={}".format(p1))

通过比较   $P((A_1A_2...A_8)|y_0)$ 和  $P((A_1A_2...A_8)|y_1)$ 的值，我们就可以判断当病人的特征为 $(A_1,A_2,...,A_8) = (1,2,...8)$ 时，该病人是否患有糖尿病。

In [ ]:
if p0>p1:
    print("该病人没有患糖尿病")
else:
    print("该病人患有糖尿病")

上面就是朴素贝叶斯诊断糖尿病的全过程。我么可以将上面判断病人是否存在糖尿病的步骤写成函数，如下：

In [ ]:
def predict(input_data):
    probalities = {}
    # 表示两个可能性 y=0 和 y = 1
    for m in range(2):
        probalities[m] = 1
        for i in range(8):
            mean = model[m][i][0]
            stdev = model[m][i][1]
            p = gaussian_probality(input_data[i], mean, stdev)
            probalities[m] = probalities[m]*p
    # 没有患糖尿病
    if(probalities[0] > probalities[1]):
        return 0
    else:
        return 1


predict(test)

最后让我们将所有的测试数据传入预测函数中，进行预测。然后根据预测结果判断该模型的准确率。

In [ ]:
count=0
for i in range(len(X_test)):
    if(predict(X_test[i])==y_test[i]):
        count = count+1
p =  count/len(X_test)  
print("该模型的预测准确率为:{:.2f}% ".format(p*100))

### 实验总结

从模型的预测准确率可以看出，虽然贝叶斯推断中含有许多的假设（比如假设连续型变量呈正太分布、$p(y=0)=p(y=1)$ 以及特征相互独立等），但是该算法对于真实数据的预测效果也还不错，能够达到 70% 左右的准确率。

<hr><div style="color: #999; font-size: 12px;"><i class="fa fa-copyright" aria-hidden="true"> 本课程内容版权归实验楼所有，禁止转载、下载及非法传播。</i></div>